### (0) Import Dependencies

In [1]:
import numpy as np
import scipy as sp

In [2]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [3]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [4]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    # NOTE(roy) don't get scared by the field/default_factory thing! It's just a way to do default field initialization for dataclasses,
    # see https://docs.python.org/3/library/dataclasses.html. Don't worry about it!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation
        # Refer to this source for derivation: https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

# Simply treat as a global
p = Params()

##### (0.5) Testing

In [5]:
# # (1) MATRIX MUlTIPLICATION EXAMPLE
# L = 1
# K = 1
# mixing_matrix = np.array([
#             [1,1,1,1],
#             [0,L,0,-L],
#             [-L,0,L,0],
#             [K,-K,K,-K]
#         ])

# V = np.array([1,1,1,1])

# X = np.matmul(mixing_matrix,V)
# X[0]
# X[1:4]
# X[0]

In [6]:
# # (2) ORIENTATION AND ACCELRATION EXAMPLE
# X = QuadrotorState([0,0,0])
# # X.orientation * np.array([0,0,2])
# R = X.orientation       # Rotation matrix
# accel = np.array([0,0,-1]) + R * np.array([0,0,5])
# accel

In [7]:
# g = p.g
# m = p.mass
# u1 = 4
# u2 = np.array([1,1,1])

# I = p.inertia
# M = u2
# w = X.angular_velocity
# I_inv = np.linalg.inv(I)
# complicated_sum = M - np.cross(w, np.matmul(I,w))
# angular_accel = np.matmul(I_inv, complicated_sum)

# angular_accel

### (1) Building a 3D quadrotor model
In this second notebook, we'll build a full 3-dimensional quadrotor model! In this one I will be leaving you guys alone a lot more, but as discussed in the lectures, there is a lot of overlap with the 2D model we made last week, so feel free to copy your implementation from there and simply expand it here!

**When implementing things like this, I recommend working in steps!** For example, in your 2D model, you may find it's much better to _first_ model the accelerations; see whether that works and only _then_ move on to the angular acceleration. If you do both at the same time, finding bugs (and everyone introduces bugs once in a while!) is much harder. **Keep testing as you write code!**

#### Recommended steps
- Copy your 2D quadrotor model (**DONE**)
- Work your way down the same way we did on the 3D model 
    - Implement the mixing matrix (i.e. compute u1 and u2 from the rotor rates) (**DONE**)
    - Modify the `state_derivative` functions to ta|ke a vector for u2 rather than a float (because now of course that's 3 numbers!) (**DONE**)
    - Modify the `accel` formulation in your model to 3D (see equations in slides)  (**DONE**)
    - Modify the `angular_accel` formulation in your model to 3D (see equations in slides)(**DONE**)

In [10]:
# NOTE(roy) feel free to copy large parts from your previous assignment!
class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self) -> None:     ## DONE!
        # NOTE(roy): I advise you to scroll through dynamics.py and see what the __init__
        # function in the base class does!
        ## Sets times steps =.01, and state to the origin
        super().__init__()

    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray) -> np.ndarray:      ## DONE!
        # NOTE(roy): Implement the simple rotor trust model we discussed in the slides
        # To see how complex this can get, see for example: https://arxiv.org/pdf/1601.00733.pdf

        # NOTE(elijah) To turn rotor_rates into thrust we use F = k_T w^2      
        # k_T = Thrust coefficient
        # w = Rotor rate
        k_T = p.k_thrust
        w =  rotor_rates
        F = k_T * np.square(w)
        return F

    def step(self, t: float, input: QuadrotorCommands) -> QuadrotorState:   ## WORK IN PROGRESS
        # NOTE(roy) Use only the first two rotor rates!
        Forces = FullQuadrotorDynamics.rotor_thrust_model(input.rotor_rates)
        L = p.arm_length
        k_t = p.k_thrust
        k_m = p.k_torque
        K = k_m/k_t


        mixing_matrix = np.array([
            [1,1,1,1],
            [0,L,0,-L],
            [-L,0,L,0],
            [K,-K,K,-K]
        ])
        U = mixing_matrix @ Forces

        # NOTE(roy) Implement the thrust/torque calculation we discussed in the slides
        
        u1 = U[0]           # Total Thrust
        u2 = U[1:4]         # Total Torque
        # raise NotImplementedError("Thrust/torque calculation not implemented!")

        # NOTE(roy) this local function is a wrapper around the class state_derivative function         ## DONE!!
        # all this does is convert the our state dataclass from/into a vector that the scipy
        # integration solver can work with. See dynamics.py for the definition of these
        # from/to state vector functions.
        def state_derivative(t: float, state: np.ndarray) -> np.ndarray:
            return FullQuadrotorDynamics.state_derivative(
                QuadrotorState.from_state_vector(state),
                u1,
                u2,
            ).to_state_vector()

        state_vector = self.state.to_state_vector()


        # raise NotImplementedError("Use the scipy function solve_ivp to integrate to the next timestep")
         # NOTE(roy) solution = sp.integrate.solve_ivp(....
        
        solution = sp.integrate.solve_ivp(
            fun= state_derivative,
            t_span = (0, self.dt),
            y0 = state_vector,
        )

        self.state = QuadrotorState.from_state_vector(solution['y'][:, -1])
        return self.state

    @staticmethod
    def state_derivative(state: QuadrotorState, u1: float, u2: np.ndarray) -> QuadrotorState:        ## WORK IN PROGRESS
        # raise NotImplementedError("State derivative not implemented!")

        # NOTE(roy) compute the relevant values here! Note that we're using only the x- and z dimensions for translation
        # and only the y-axis for rotation for this planar model.
        # accel = np.zeros(3)
        g = p.g
        m = p.mass

        R = state.orientation     # Rotation matrix
        accel = np.array([0,0,-g]) +  R * np.array([0,0,u1/m]) # np.array([0,0,-g]) +  R * np.array([0,0,u1/m])

        I = p.inertia
        M = u2
        w = state.angular_velocity
        I_inv = np.linalg.inv(I)
        complicated_sum = M - np.cross(w, np.matmul(I,w))
        angular_accel = np.matmul(I_inv, complicated_sum)

        # NOTE(roy) this might look a little confusing at first glance, but we're populating DERIVATIVES
        # of the state here! So position -> velocity, velocity -> acceleration, etc...
        # If you can think of a more readable way to do this (e.g. defining a QuadrotorStateDerivative class)
        # feel free to open a pull-request into the upstream repository!
        state_derivative = QuadrotorState(
            position=state.velocity,
            velocity=accel,
            orientation=dRot3(state.orientation, state.angular_velocity),
            angular_velocity=angular_accel,
        )

        return state_derivative

This code is largely the same, I set the rotor rates to 300 rad/s now, you should see it be very close to hovering with these numbers! (As I hope you found in the previous assignment as well with 600rad/s!)

In [16]:
print("Thrust Coefficient", p.k_thrust)
print("Upward Force:", 4 * p.k_thrust* 300**2 )
print("Gravity Force:", p.mass * p.g)
hover_rate = np.sqrt(   (p.mass * p.g) / (4 * p.k_thrust))
print("hover_rate:", hover_rate)

Thrust Coefficient 1.8795926921113627e-05
Upward Force: 6.766533691600906
Gravity Force: 13.2585908
hover_rate: 419.9395843340998


In [17]:
class EmptyTrajectory(TrajectoryBase):
    def eval(self, *args):
        return TrajectoryState(0.0, position=np.zeros(3))  # Trajectory not yet important

class EmptyController(ControllerBase):
    def step(self, *args) -> QuadrotorCommands:
        return QuadrotorCommands(hover_rate * np.ones(4))

In [18]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d, animate_matplotlib

In [71]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 0.5):
    sim = SimulatorBase(
        dt=0.01,
        dynamics=FullQuadrotorDynamics(),
        controller=ForwardsBackwards(),
        trajectory=EmptyTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [72]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.array([0,0,0]),
)

run_and_render(initial_state, 5)

Output()

### Does the simulated model make any sense?
If all is well, you should see the quadrotor roughly hovering with those 429 rad/s rotor rates

Now try some experiments for yourself and describe whether the behavior is as expected or whether there is anything wrong with your model, ideas to try:
- Can you make the vehicle yaw by setting a combination of rotor rates?
- Can you make the vehicle move forward/backward left/right by correctly lowering two rotor rates each time?
- What happens if you're in hover (i.e. the 429rad/s) and disable one rotor?

In [73]:
# (0) Getting the vehicle to hover
hover_rate = np.sqrt((p.mass * p.g) / (4 * p.k_thrust))
HOVER_RATES = np.ones(4) * hover_rate

# (1) Getting the vehicle to yaw
m = p.mass
g = p.g 
k = p.k_thrust
s = 1  # Difference between rotor rates
r = ((m*g) / (4*k) - s**2)**(1/2)
ROTOR_RATES = np.array([r + s, r - s, r + s, r -s])
N_ROTOR_RATES = np.array([r-s, r+s, r-s, r+s])
print(r)
print(ROTOR_RATES)

class YawController(ControllerBase):
    def step(self, *args) -> QuadrotorCommands:
        t = args[0]
        if t // 3 < 1:
            return QuadrotorCommands(ROTOR_RATES)
        elif t // 3 < 3: 
            return QuadrotorCommands(N_ROTOR_RATES)
        else: 
            return QuadrotorCommands(HOVER_RATES)
        

# (2) Get vehicle to move forward and thne backwards
m = .001
FORWARD_RATES = np.array([r - s, r - s, r , r ])
BACKWARD_RATES = (1+m) * np.array([r + s, r + s , r , r ])
class ForwardsBackwards(ControllerBase):
    def step(self, *args) -> QuadrotorCommands:
        t = args[0]
        if t < .5:
            return QuadrotorCommands(FORWARD_RATES)
        elif t < 2 : 
            return QuadrotorCommands(BACKWARD_RATES)
        elif t < 3: 
            return QuadrotorCommands(1.1* FORWARD_RATES)
        else: 
            return QuadrotorCommands(FORWARD_RATES)
        

# (3) Hover and disable one rotor

class CatastropheController(ControllerBase):

    def step(self, *args) -> QuadrotorCommands:
        t = args[0]
        if t < 2.5:
            return QuadrotorCommands(HOVER_RATES)
        else:
            DISABLED_RATES = HOVER_RATES.copy()
            DISABLED_RATES[3] -= .2*t
            return QuadrotorCommands(DISABLED_RATES)
        
print(HOVER_RATES)

419.9383936849505
[420.93839368 418.93839368 420.93839368 418.93839368]
[419.93958433 419.93958433 419.93958433 419.93958433]


### Commit your updated notebook to your fork!